In [1]:
import rdflib

INFO:rdflib:RDFLib Version: 4.2.1


In [4]:
g=rdflib.Graph()

In [7]:
def extract_data(graph):
    new_graph=rdflib.Graph()
    for s,p,o in g:
        new_graph.add((s,p,o))
        subject_graph = rdflib.Graph()
        # For our subjects we extract type assertions from dbpedia ontology
        subject_graph.load(s)
        for s2,p2,o2 in subject_graph: 
            if p2.encode('utf-8') == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type' and 'dbpedia.org/ontology' in o2.encode('utf-8'):
                new_graph.add((s2,p2,o2))
        obj_graph = rdflib.Graph()
        # For our subjects we extract type assertions from dbpedia ontology
        obj_graph.load(o)
        for s2,p2,o2 in obj_graph: 
            if p2.encode('utf-8') == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type' and 'dbpedia.org/ontology' in o2.encode('utf-8'):
                new_graph.add((s2,p2,o2))
        
    return new_graph

def gen_paths(graph,node,depth,max_depth,all_paths,path):
    #print('current depth:',depth,' max depth: ',max_depth)
    #print('node: ', node)
    if depth<max_depth:
        if  'dbpedia.org/ontology' in node.encode('utf-8') or 'dbpedia.org/resource' in node.encode('utf-8'):
            for p,o in graph.predicate_objects(node):               
                if p.encode('utf-8') == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type' and 'dbpedia.org/ontology' in o.encode('utf-8'):
                    if len(path) == 0:
                        #print('  found new subj: ', 'subj_'+str(o))
                        all_paths.append('subj_'+str(o))
                    else:
                        new_path=path+'_'+str(o)
                        #print('  found new extended type: ',new_path)
                        all_paths.append(new_path)
                else:
                    #print('    extending: ',path+'_'+p)
                    gen_paths(graph,o,depth+1,max_depth,all_paths,path+'_'+str(p))
    else:
        all_paths.append(path)
        

In [11]:
g.load('http://swat.cse.lehigh.edu/projects/lubm/University0_0.owl', format='xml')
all_paths = []
# ext_data = extract_data(g)
for s,p,o in g:
    gen_paths(g,s,0,2,all_paths,'')

In [12]:
print(len(all_paths))


0
